# Swim Transformer : Hierarchical Vision Transformer using Shifted Windows

在2021年微软亚洲研究院基于Transformer架构提出了一种通用的视觉backnone，使用分层窗口注意力机制移位窗口注意力计算等方法解决了原有的Transformer因为模型庞大难以应用在目标检测和语义分割这种像素级任务上的缺陷。

目前Transformer应用到图像领域主要有两大挑战：

- 视觉实体变化大，在不同场景下视觉Transformer性能未必很好
- 图像分辨率高，像素点多，Transformer基于全局自注意力的计算导致计算量较大

针对上述两个问题，我们提出了一种**包含滑窗操作，具有层级设计**的Swin Transformer。

其中滑窗操作包括**不重叠的local window，和重叠的cross-window**。将注意力计算限制在一个窗口中，**一方面能引入CNN卷积操作的局部性，另一方面能节省计算量。**

## 模型架构


<div align = "center">
    <img src = ./images/swim-tfm-0.png width = 90%>
</div>

整个模型采取层次化的设计，一共包含4个Stage，每个stage都会缩小输入特征图的分辨率，翻倍通道数，像CNN一样逐层扩大感受野。

与Vit相比，Swim-tfm有以下几点区别：

- 在模型开头取消Position embedding和Class token的操作，直接进行patch分割
- 从Stage2开始，最开始都进行Patch Merging，用于降低图片分辨率
- MSA修改为Window MSA以及Shifted window MSA，且先经过W-MSA再经过SW-MSA

参考[微软官方代码](https://github.com/microsoft/Swin-Transformer)，下面是Swim-tfm实现的总体流程。

```python
class SwinTransformer(nn.Module):
    def __init__(...):
        super().__init__()
        ...
        # absolute position embedding
        if self.ape:
            self.absolute_pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
            
        self.pos_drop = nn.Dropout(p=drop_rate)

        # build layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(...)
            self.layers.append(layer)

        self.norm = norm_layer(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        x = self.patch_embed(x)
        if self.ape:
            x = x + self.absolute_pos_embed
        x = self.pos_drop(x)

        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)  # B L C
        x = self.avgpool(x.transpose(1, 2))  # B C 1
        x = torch.flatten(x, 1)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x
```

接下来再来看各个组件的实现细节

## Patch Embedding

在整个Swim-tfm的流程中，第一步就是划分图像为数个Patch，之后再嵌入向量。

具体做法为使用 **2d卷积** 对图像进行patch划分，然后进行Flatten操作进行向量嵌入。

In [11]:
import torch
import torch.nn as nn

def to_tuple(t):
    return t if isinstance(t, tuple) else (t, t)


class Patch_Embedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embedding_dim, norm_layer=None):
        super(Patch_Embedding, self).__init__()
        image_size = to_tuple(image_size)
        patch_size = to_tuple(patch_size)
        patch_number = (image_size[0] // patch_size[0]) * (image_size[1] // patch_size[1])
        self.image_size = image_size
        self.patch_size = patch_size
        self.patch_number = patch_number

        self.in_channels = in_channels
        self.embedding_dim = embedding_dim

        self.projection = nn.Conv2d(in_channels, embedding_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm_layer = norm_layer(embedding_dim)
        else:
            self.norm_layer = None

    def forward(self, x):
        x = self.projection(x)
        print("dim after projection : {} ".format(x.shape))
        x = torch.flatten(x, 2) # n c h w -> n c h*w
        x = torch.transpose(x, 1, 2) # n c h*c -> n h*w c
        if self.norm_layer is not None:
            x = self.norm_layer(x)
        return x


if __name__ == "__main__":
    PatchEmbedding = Patch_Embedding(image_size=224,
                                     patch_size=4,
                                     in_channels=3,
                                     embedding_dim=96)
    
    img = torch.randn(1, 3, 224, 224)

    result = PatchEmbedding(img)
    print(result.shape)

dim after projection : torch.Size([1, 96, 56, 56]) 
torch.Size([1, 3136, 96])


## Patch Merging

该模块的作用是在每个Stage开始前做降采样，用于缩小分辨率，调整通道数 进而形成层次化的设计，同时也能节省一定运算量。

> 在CNN中，则是在每个Stage开始前用stride=2的卷积/池化层来降低分辨率。
> 每次降采样是两倍，因此在行方向和列方向上，间隔2选取元素。

然后拼接在一起作为一整个张量，最后展开。**此时通道维度会变成原先的4倍（因为H,W各缩小2倍），此时再通过一个全连接层再调整通道维度为原来的两倍**

In [14]:
class Patch_Merging(nn.Module):
    def __init__(self, input_resolution, dim, norm_layer=None):
        super(Patch_Merging, self).__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        if norm_layer is not None:
            self.norm_layer = norm_layer
        else:
            self.norm_layer = nn.LayerNorm(4 * dim)

    def forward(self, x):
        '''
        x : b h*w c
        '''
        H, W  = self.input_resolution
        B, L, C = x.shape
        assert H*W == L, "input features has wrong size."
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :] # b, h/2, w/2, c
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]

        x = torch.cat([x0, x1, x2, x3], -1) # b, h/2, w/2, 4*c
        x = self.norm_layer(x)
        x = self.reduction(x)
        return x


if __name__ == "__main__":
    # middle_feature_map = torch.randn(1, 3136, 96)

    PatchMerging = Patch_Merging(input_resolution=[56, 56],
                                 dim=96)

    result = PatchMerging(result)
    print(result.shape)

torch.Size([1, 28, 28, 192])


上述流程示意图如下：

<div align = "center">
    <img src = ./images/swim-tfm-1.png width = 90%>
</div>

## Window Partition and Reverse

window partition函数是用于对张量划分窗口，指定窗口大小。将原本的张量从 N H W C, 划分成 num_windows * B, window_size, window_size, C，其中 num_windows = H * W / (window_size * window_size)，即窗口的个数。而window reverse函数则是对应的逆过程。这两个函数会在后面的Window Attention用到。

In [1]:
def window_partition(x, window_size):
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    # permute方法用于对张量维度进行重新排序（这一过程不会复制张量，因此属于是一种高效的方法） contiguous方法用于确保所处理的张量在内存中的连续性
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C) 
    return windows

def window_reverse(windows, window_size, H, W):
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x

## Window Attention

传统的Transformer都是基于全局来计算注意力的，因此计算负责度十分高，而Swim-tfm则将计算注意力这一个过程限制在一个窗口之内，进而减少了计算量。

Swim-tfm计算attention的公式如下：

$$Attention(Q,K,V) = Softmax (\frac{Q K^{T}}{\sqrt{d}} + B)V$$

和之前Transformer计算attention的区别在于加入了 **B** 其表示相对位置编码，在原论文的实验中发现，相对位置编码的加入提升了模型性能。

这部分代码的讲解参考[图解Swim-tfm](https://zhuanlan.zhihu.com/p/367111046)

In [ ]:
class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads # nH
        head_dim = dim // num_heads # 每个注意力头对应的通道数
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 设置一个形状为（2*(Wh-1) * 2*(Ww-1), nH）的可学习变量，用于后续的位置编码
  
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)
     # 相关位置编码...
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.meshgrid([coords_h, coords_w]) # -> 2*(wh, ww)
        coords = torch.stack(coords)  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords_first = coords_flatten[:, :, None]  # 2, wh*ww, 1
        relative_coords_second = coords_flatten[:, None, :] # 2, 1, wh*ww
        relative_coords = relative_coords_first - relative_coords_second # 最终得到 2, wh*ww, wh*ww 形状的张量
        relative_coords = relative_coords.permute(1, 2, 0).contiguous() # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0) # (1, num_heads, windowsize, windowsize)

        if mask is not None: # 下文会分析到
            ...
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

- 首先输入张量形状为 （numWindows * B, window_size * window_size, C）（后续会解释）
- 然后经过self.qkv这个全连接层后，进行reshape，调整轴的顺序，得到形状为（3, numWindows * B, num_heads, window_size * window_size, c//num_heads），并分配给q,k,v。
- 根据公式，我们对q乘以一个scale缩放系数，然后与k（为了满足矩阵乘要求，需要将最后两个维度调换）进行相乘。得到形状为(numWindows * B, num_heads window_size * window_size, window_size * window_size)的attn张量
- 之前我们针对位置编码设置了个形状为(2 * window_size-1 * 2 * window_size-1, numHeads)的可学习变量。我们用计算得到的相对编码位置索引self.relative_position_index选取，得到形状为(window_size * window_size, window_size * window_size, numHeads)的编码，加到attn张量上
- 暂不考虑mask的情况，剩下就是跟transformer一样的softmax，dropout，与V矩阵乘，再经过一层全连接层和dropout

## Shifted Window Attention

前面的Window Attention是在每个窗口下计算注意力的，为了更好的和其他window进行信息交互，Swin Transformer还引入了shifted window操作。

<div align = "center">
    <img src = ./images/swim-tfm-2.png width = 90%>
</div>

左边是没有重叠的Window Attention，而右边则是将窗口进行移位的Shift Window Attention。可以看到移位后的窗口包含了原本相邻窗口的元素。但这也引入了一个新问题，即window的个数翻倍了，由原本四个窗口变成了9个窗口。

在实际代码里，我们是通过对特征图移位，并给Attention设置mask来间接实现的。能在保持原有的window个数下，最后的计算结果等价。

<div align = "center">
    <img src = ./images/swim-tfm-3.png width = 90%>
</div>

## 特征图移位操作

代码里对特征图移位是通过torch.roll来实现的，简单理解：shifts的值为正数相当于向下挤牙膏，挤出的牙膏又从顶部塞回牙膏里面；shifts的值为负数相当于向上挤牙膏，挤出的牙膏又从底部塞回牙膏里面.

<div align = "center">
    <img src = ./images/swim-tfm-4.png width = 90%>
</div>

## Attention Mask

我认为这是Swin Transformer的精华，通过设置合理的mask，让Shifted Window Attention在与Window Attention相同的窗口个数下，达到等价的计算结果。

首先我们对Shift Window后的每个窗口都给上index，并且做一个roll操作，即shifts值为负数，先对行后对列做roll操作。

<div align = "center">
    <img src = ./images/swim-tfm-5.png width = 90%>
</div>

我们希望在计算Attention的时候，让具有相同index QK进行计算，而忽略不同index QK计算结果。

最后正确的结果如下图所示 (PS: 这个图的Query Key画反了。。。应该是4x1 和 1x4 做矩阵乘）

<div align = "center">
    <img src = ./images/swim-tfm-6.png width = 90%>
</div>

而要想在原始四个窗口下得到正确的结果，我们就必须给Attention的结果加入一个mask（如上图最右边所示）

相关代码如下：

```python
     if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
```

以上图的设置，我们用这段代码会得到这样的一个mask

```python
tensor([[[[[   0.,    0.,    0.,    0.],
           [   0.,    0.,    0.,    0.],
           [   0.,    0.,    0.,    0.],
           [   0.,    0.,    0.,    0.]]],


         [[[   0., -100.,    0., -100.],
           [-100.,    0., -100.,    0.],
           [   0., -100.,    0., -100.],
           [-100.,    0., -100.,    0.]]],


         [[[   0.,    0., -100., -100.],
           [   0.,    0., -100., -100.],
           [-100., -100.,    0.,    0.],
           [-100., -100.,    0.,    0.]]],


         [[[   0., -100., -100., -100.],
           [-100.,    0., -100., -100.],
           [-100., -100.,    0., -100.],
           [-100., -100., -100.,    0.]]]]])
```

在之前的window attention模块的前向代码里，包含这么一段

```python
    if mask is not None:
        nW = mask.shape[0]
        attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
        attn = attn.view(-1, self.num_heads, N, N)
        attn = self.softmax(attn)
```

将mask加到attention的计算结果，并进行softmax。mask的值设置为-100，softmax后就会忽略掉对应的值

## Transformer Block整体架构

两个连续的Block架构，需要注意的是一个Stage包含的Block个数必须是偶数，因为需要交替包含一个含有Window Attention的Layer和含有Shifted Window Attention的Layer。

我们看下Block的前向代码

```python
    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x
```

整体流程如下

- 先对特征图进行LayerNorm
- 通过self.shift_size决定是否需要对特征图进行shift
- 然后将特征图切成一个个窗口
- 计算Attention，通过self.attn_mask来区分Window Attention还是Shift Window Attention
- 将各个窗口合并回来
- 如果之前有做shift操作，此时进行reverse shift，把之前的shift操作恢复
- 做dropout和残差连接
- 再通过一层LayerNorm+全连接层，以及dropout和残差连接
